## Classifying letters

This notebook contains a training script for a classifier that can recognize a few letters from a bitmap.
Version 2: uses data from the wand to validate ("transfer learning", if you will)

In [58]:
from datetime import datetime

import numpy as np
from matplotlib import pyplot
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import tensorflow as tf
import tensorflow_datasets as tfds
import pathlib
import shutil
from os import path
from pathlib import Path

from PIL import Image

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


create new dataset from wand drawn images

In [ ]:
# first, copy all images of same class to single dir

def moveWandFiles(source_folder:pathlib.Path, target_folder:pathlib.Path,letter):
    target_folder= os.path.join(target_folder + letter )   
    source_folder= os.path.join(source_folder + letter )   
    Path(target_folder).mkdir(parents=True, exist_ok=True)
    ii = 1
    for image_file in Path(source_folder).rglob("*.bmp"): # recursively find image paths
        # Separate base from extension
        base, extension = os.path.splitext(image_file.name)        
        new_name = os.path.join(letter + "_" + str(ii) + extension)    
        shutil.copy(image_file, Path(target_folder).joinpath(new_name))
        ii += 1


    
letter = 'L'        
OG_lettersInD = 'C:\\Users\\john\\Documents\\Arduino\\priWand\\letters\\'      
destCollated = 'C:\\Users\\john\\Documents\\Arduino\\priWand\\letters\\allWandLetters\\'  

rootimage = OG_lettersInD
disdir = destCollated     

moveWandFiles(rootimage, disdir,letter)
        
tf.keras.preprocessing.image_dataset_from_directory(
    destCollated,
    labels="inferred",
    label_mode="int",
    class_names=letter,
    color_mode="grayscale",
    batch_size=32,
    image_size=(28, 28),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
)

In [69]:
# Set of letters we want to recognized
_LETTERS = 'MOCL'
# A set of letters used to train the "other" category. These shouldn't look
# similar to any of the letters in _LETTERS
_OTHER_LETTERS = 'FHIKRTY'


# Labels in the emnist/letters dataset that should be used for training.
_LABELS = [ord(c) - ord('A') + 1 for c in _LETTERS + _OTHER_LETTERS]

_WANTED_LABELS = [ord(c) - ord('A') + 1 for c in _LETTERS]
_OTHER_LABEL = 0
_NUM_CLASSES = len(_LETTERS) + 1  # All "other" letters classified as 0.
_BATCH_SIZE = 32

# load train and test dataset



def load_dataset():
    # load dataset
    (train_ds, test_ds), ds_info = tfds.load(
        name='emnist/letters',
        split=['train', 'test'],
        shuffle_files=True,
        with_info=True,
        as_supervised=True,
        decoders={
            # Don't decode images, the dataset will get filtered,
            # and we shouldn't decode what we don't use.
            'image': tfds.decode.SkipDecoding(),
        })

    train_ds = prepare(train_ds, ds_info)
    test_ds = prepare(test_ds, ds_info)

    return (train_ds, test_ds), ds_info


def prepare(dataset, ds_info):
    labels = tf.constant(_LABELS, dtype=tf.int64)
    wanted_labels = tf.constant(_WANTED_LABELS, dtype=tf.int64)

    @tf.function
    def is_wanted(image, label):
        """Returns true if label is in _LABELS"""
        del image  # unused
        return tf.math.reduce_any(label == labels)

    @tf.function
    def map_entry(image, label):
        """Transforms the image into the form our classifier will expect."""
        decoded = ds_info.features['image'].decode_example(image)
        # Convert image to floats
        image = tf.cast(decoded, tf.float32) / 255
        # Images in emnist are transposed. Bring them back into normal direction.
        image = tf.transpose(image, perm=[1, 0, 2])
        label = tf.cond(
            tf.math.reduce_any(label == wanted_labels),
            # Relabel entries that are in _LETTERS to the range [1, len(_LETTERS)]
            lambda: tf.argmax(tf.equal(wanted_labels, label)) + 1,
            # Relabel entries in _OTHER_LETTERS to 0.
            lambda: tf.constant(_OTHER_LABEL, dtype=tf.int64))
        return image, label

    return (dataset.filter(is_wanted).cache().shuffle(1000).map(
        map_entry).batch(_BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE))


def define_model():
    # Train with images that randomly rotated, in any direction. This is
    # because we can't tell the direction the wand is held.
    # If we could, we might get better classification by reducing the range of
    # random rotation.
    factor=0.2
    data_augmentation = tf.keras.Sequential([
        layers.experimental.preprocessing.RandomRotation(
            factor, interpolation='nearest'),
    ])
    model = Sequential()
    model.add(data_augmentation)
    model.add(
        layers.Conv2D(16, (3, 3),
                      activation='relu',
                      kernel_initializer='he_uniform',
                      input_shape=(28, 28, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(
        layers.Conv2D(32, (3, 3),
                      activation='relu',
                      kernel_initializer='he_uniform'))
    model.add(
        layers.Conv2D(32, (3, 3),
                      activation='relu',
                      kernel_initializer='he_uniform'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(
        layers.Dense(50, activation='relu', kernel_initializer='he_uniform'))
    model.add(layers.Dense(_NUM_CLASSES))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(0.001),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'],
    )
    return model


# trains a model


def train_model(train_ds, test_ds):
    """Trains the model, and outputs evaluation stats to TensorBoard."""
    model = define_model()
    logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
    tboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logs,
                                                     histogram_freq=1,
                                                     profile_batch='500,520')
    # fit model
    history = model.fit(train_ds,
                        epochs=10,
                        validation_data=test_ds,
                        callbacks=[tboard_callback],
                        class_weight=_label_weights(len(_LABELS),
                                                    _WANTED_LABELS))
    # evaluate model
    _, acc = model.evaluate(test_ds)
    print('> %.3f' % (acc * 100.0))
    return model, acc, history


# plot diagnostic learning curves


def summarize_diagnostics(histories):
    for i in range(len(histories)):
        # plot loss
        pyplot.subplot(2, 1, 1)
        pyplot.title('Cross Entropy Loss')
        pyplot.plot(histories[i].history['loss'], color='blue', label='train')
        pyplot.plot(histories[i].history['val_loss'],
                    color='orange',
                    label='test')
        # plot accuracy
        pyplot.subplot(2, 1, 2)
        pyplot.title('Classification Accuracy')
        pyplot.plot(histories[i].history['accuracy'],
                    color='blue',
                    label='train')
        pyplot.plot(histories[i].history['val_accuracy'],
                    color='orange',
                    label='test')
    pyplot.show()


# summarize model performance


def summarize_performance(scores):
    # print summary
    print('Accuracy: mean=%.3f std=%.3f, n=%d' %
          (np.mean(scores) * 100, np.std(scores) * 100, len(scores)))
    # box and whisker plots of results
    pyplot.boxplot(scores)
    pyplot.show()


# run the test harness for evaluating a model


def run_training():
    """Runs training and shows learning curves."""
    # load dataset
    (train_ds, test_ds), ds_info = load_dataset()
    # evaluate model
    model, score, history = train_model(train_ds, test_ds)
    # learning curves
    summarize_diagnostics([history])
    # summarize estimated performance
    summarize_performance([score])
    return model


def _label_weights(total_num_labels, wanted_labels):
    """Reweight the label costs to account over represented OTHER label."""
    label_weights = {
        l: 1.0 / (len(wanted_labels) + 1)
        for l in range(len(wanted_labels) + 1)
    }
    label_weights[_OTHER_LABEL] = (1.0 /
                                   ((len(wanted_labels) + 1) *
                                    (total_num_labels - len(wanted_labels))))
    return label_weights


def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()


def blur(img_array):
    kernel = np.array([1, 3, 1])
    img_array = np.apply_along_axis(
        lambda x: np.convolve(x, kernel, mode='same'), 0, img_array)
    img_array = np.apply_along_axis(
        lambda x: np.convolve(x, kernel, mode='same'), 1, img_array)
    return img_array


In [70]:
model = run_training()

Epoch 1/10
    142/Unknown - 3s 14ms/step - loss: 0.0718 - accuracy: 0.5720

KeyboardInterrupt: 

In [ ]:
%tensorboard --logdir=logs

# python -m tensorboard.main --logdir=logs  


In [ ]:
# Export the model to tflite.

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
# open('model.tflite', 'wb').write(tflite_model)

modelSaveName = _LETTERS + '_'+ _OTHER_LETTERS+'.tflite'
open(modelSaveName, 'wb').write(tflite_model)

In [ ]:
# Reload the tflite model


interpreter = tf.lite.Interpreter(modelSaveName)
# interpreter = tf.lite.Interpreter('model.tflite')
interpreter.allocate_tensors()
input_tensor = interpreter.tensor(interpreter.get_input_details()[0]["index"])
output_tensor = interpreter.tensor(interpreter.get_output_details()[0]["index"])
def classify(letter_image):
    input_tensor()[:] = letter_image
    interpreter.invoke()
    probabilities = softmax(output_tensor()[0])
    index = np.argmax(probabilities)
    if not index:
        return 'UNKNOWN', probabilities[index]
    return _LETTERS[index - 1],  probabilities[index]


In [ ]:
# Run the classifier on a few examples

print('M letters')
for i in range(3, 10):
    image = Image.open(f"evaluation/{i}_M.bmp")
    image = image.crop((0, 0, 28, 28))
    np_image = np.array(image, np.float32)
    np_image = np_image / 255.0
    # Apply a blur to the input image to look more like the emnist training set.
    np_image = blur(np_image)

    pyplot.imshow(np_image, cmap='gray')
    print(i, classify(np_image.reshape((1, 28, 28, 1))))
    del image

print('O letters')
for i in range(3, 21):
    image = Image.open(f"evaluation/{i}_O.bmp")
    image = image.crop((0, 0, 28, 28))
    np_image = np.array(image, np.float32)
    np_image = np_image / 255.0
    np_image = blur(np_image)
    np_image = np_image / np.max(np_image)

    pyplot.imshow(np_image, cmap='gray')
    print(i, classify(np_image.reshape((1, 28, 28, 1))))
    del image
